<h2> Code to organize data so that it can be processed in // Bohrer and Larson 2022 <\h2>

In [ ]:
#First we are going to load in the needed programs, if you dont have them you will have to get them
import numpy as np
import pickle 
pickle.HIGHEST_PROTOCOL = 4
import pandas as pd
from scipy.stats import norm, gaussian_kde
from scipy.spatial.distance import pdist
from scipy.spatial.distance import cdist
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
import copy
import seaborn as sns
import dill #This will be used to store the workplace so you dont have to run certain cells
sns.set()
import os 
from math import nan
import math 
import ast

In [ ]:
#First we are going to link the individual chromosomes with their transcription states so it is easier to handle
#This is what we mean by linked trajectories.


try: 
    os.makedirs('Linked_Trajectories') 
except:
    print('File is allready there Chris!')

#Load in the data used, will be over 7000 chromosomes 
filename="chromosome21.tsv"
df_temp = pd.read_csv(filename,sep="\t")


#Here is the actual code, very straight forward, this is for the dis
ars=df_temp['Chromosome copy number'].to_numpy()
xx=df_temp['X(nm)'].to_numpy()
yy=df_temp['Y(nm)'].to_numpy()
zz=df_temp['Z(nm)'].to_numpy()
index=df_temp['Genomic coordinate'].to_numpy()

Trans_state=df_temp['Transcription'].to_numpy()
Check=np.unique(index)


for kcat in np.unique(ars):
    
    Cords=np.zeros(len(Check))
    inds=df_temp['Chromosome copy number']==kcat
    
    if len(index[inds])==len(Cords):
        
        Cords=np.transpose([xx[inds], yy[inds], zz[inds]])
        
        file = "Linked_Trajectories/Test"+str(kcat)
        np.save(file, Cords)  
        
    else:
        
        print('problem')
        
        

        
        

        
        
        
        
#This is going to link together the corresponding transcription trajectories, it is also going to grab out the gene names
#Note, if there are multiple genes per a locus, then we only take the first one so that we do not over weight that locus. 
        
        
#Make the directories if they are not there
try: 
    os.makedirs('Transcription_Trajectories') 
except:
    print('File is allready there Chris!')
    
try: 
    os.makedirs('Gene_Names') 
except:
    print('File is allready there Chris!')


ars=df_temp['Chromosome copy number'].to_numpy()
xx=df_temp['X(nm)'].to_numpy()
yy=df_temp['Y(nm)'].to_numpy()
zz=df_temp['Z(nm)'].to_numpy()
index=df_temp['Genomic coordinate'].to_numpy()

Trans_state=df_temp['Transcription'].to_numpy()
Check=np.unique(index)

Gene_Names=df_temp['Gene names'].to_numpy()


(_, _, filenames) = next(os.walk('Linked_Trajectories'))

new_lined_up_ind=[]
for kcat in range(len(filenames)):
    filenames_2=filenames[kcat]
    new_lined_up_ind.append(int(filenames_2[4:-4]))


for i in range(len(Check)):
    print(i)

    Cords3=np.zeros(len(np.unique(ars)))
    Gene_Names_to_Save=np.zeros(len(np.unique(ars)))
    
    count=0
    for kcat in range(len(new_lined_up_ind)):
        
        
        #Little different than before, but will line up correctly
        inds=df_temp['Chromosome copy number']==kcat+1


        Cords2=Trans_state[inds]
        Gene_Names_Temp=Gene_Names[inds]
        
        try:
            np.isnan(Cords2[i])
            Cords3[count]=nan
            Gene_Name_Temper=nan
        except:
            tempstate=Cords2[i]
            Gene_Name_Temper=Gene_Names_Temp[i]

            if tempstate[0:2]=='on':
                Cords3[count]=1
            elif tempstate[0:2]=='of':
                Cords3[count]=0
            else:
                Cords3[count]=nan
            
        count=count+1

   
    file = "Transcription_Trajectories/Test"+str(i)
    np.save(file, Cords3)
    
    file = "Gene_Names/Test"+str(i)
    np.save(file, Gene_Name_Temper)


<h2> Gathering Distances in individual files </h2>

In [ ]:
#Here is the code for generating the individual distance files. Aka for the distances from one locus to all others for different chromosomes. 
#The way this was set up was for on a cluster. It is easy to code this so if you want to run it, you might as well write your own version. 
#The results of this analysis are in the folder 'Distances'

import os
from multiprocessing import Pool
import numpy as np
import pickle 
pickle.HIGHEST_PROTOCOL = 4
import pandas as pd
from scipy.stats import norm, gaussian_kde
from scipy.spatial.distance import pdist
from scipy.spatial.distance import cdist
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
import copy
import seaborn as sns
from math import nan
import dill #This will be used to store the workplace so you dont have to run certain cells
import NPEETmaster.npeet.entropy_estimators as ee
sns.set()
from scipy.stats import entropy
from pathlib import Path
from math import nan
import signal



def init_worker():
    """
    This is necessary to be able to interrupt the
    script with CTRL-C (or scancel for that matter).
    It makes sure that the workers ignore SIGINT so
    that any SIGINT sent goes to the master process
    """
    signal.signal(signal.SIGINT, signal.SIG_IGN)


def worker(kcat):
        print('Here we go',kcat)
        
        Data_Folder='/data/bohrerch/'
        #Data_Folder=''
        areol=np.random.randint(0,100)+kcat
        np.random.seed(seed=areol)
        
        for ksksks in range(1000):
            num_of_loci=500
            (_, _, filenames) = next(os.walk(Data_Folder+'Linked_Trajectories'))
            Cor=np.load(Data_Folder+'Linked_Trajectories/'+filenames[0])
            num_of_loci=len(Cor)
            barcode1=np.random.randint(0,num_of_loci)
            
            file = Path(Data_Folder+"Distances/Distances_"+str(barcode1))
            print('Im inside')
            
            if file.is_file()==0:
                df = pd.DataFrame();
                for barcode2 in range(num_of_loci):
                    df1 = pd.DataFrame();

                    print(barcode1/num_of_loci, barcode2/num_of_loci, kcat)

                    distances=[]
                    count=0
                    for kcat2 in range(len(filenames)):

                        file = Data_Folder+"Linked_Trajectories/Test"+str(kcat2+1)+'.npy'
                        Cor=np.load(file)

                        dis=pdist(np.vstack((Cor[barcode1,:],Cor[barcode2,:])))
                        newdis=dis[0]
                        
                        distances.append(newdis)
                        

                    df1["barcode"+str(barcode2)]=np.abs(distances)
                    df=pd.concat([df,df1], axis=1)

                file = Data_Folder+"Distances/Distances_"+str(barcode1)

                df.to_csv(file,index=False)
                
            
        
        





if __name__ == '__main__':
    nproc = int(os.environ.get("SLURM_CPUS_PER_TASK", "2"))
    print("Running on %d CPUs" % nproc)
    
    p = Pool(nproc, init_worker)
    try:
                # the the number of allocated cpus (or 2 if not run as a slurm job)
               

               

                num_of_loci=1600


                tasks = range(0, 100)
                print('Hey Im starting')                    # Create a multiprocessing Pool
                p.map(worker, tasks) 



    except (KeyboardInterrupt, SystemExit):
            p.terminate()
            p.join()
            sys.exit(1)
    else:
            p.close()
            p.join()
            # result summary
            #print("\n".join("%d * %d = %d" % (a, a, b) for a, b in zip(tasks, results)))


